In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Bahia - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Bahia - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Amazonas - Desemprego,Bahia - Consumo de Cimento (t)
0,2003,483.900236,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,24.300584,...,73.888459,1.602501,0.669899,3.692839,1.317344e+08,8.384593e+06,8.566149,1.216359e+08,8.630942,1658.094
1,2004,522.456171,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,21.294803,...,71.475857,1.699240,0.673628,3.775586,1.336785e+08,8.472829e+06,8.602876,1.223016e+08,8.558110,1733.557
2,2005,569.481605,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,19.335488,...,71.485298,1.805776,0.677007,3.824208,1.354495e+08,8.522905e+06,8.631972,1.228206e+08,8.526727,1668.001
3,2006,633.857084,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,18.298188,...,75.718102,1.930499,0.680049,3.940207,1.370423e+08,8.592312e+06,8.653845,1.232167e+08,8.446276,1934.967
4,2007,731.905950,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,16.610190,...,78.437621,1.812937,0.682761,4.044809,1.385256e+08,8.640798e+06,8.676548,1.235991e+08,8.434407,2259.804
5,2008,874.866956,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,19.314465,...,77.493797,1.586819,0.685154,4.100371,1.396949e+08,8.691699e+06,8.687931,1.237178e+08,8.396886,2678.337
6,2009,1028.843987,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,20.202505,...,78.022515,1.421020,0.687239,4.185427,1.406176e+08,8.722094e+06,8.695651,1.237698e+08,8.420659,2825.491
7,2010,1146.660535,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,20.155807,...,78.179391,1.422041,0.689024,4.338533,1.439300e+08,8.745548e+06,8.730895,1.244348e+08,8.389077,3349.109
8,2011,1256.520719,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,20.048279,...,77.933380,1.514544,0.690520,4.536829,1.469771e+08,8.709535e+06,8.759313,1.249553e+08,8.376520,3615.893
9,2012,1339.122148,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,20.344237,...,78.542856,1.555217,0.691525,4.749958,1.498366e+08,8.659814e+06,8.775083,1.254646e+08,11.321769,3744.322


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Bahia - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Amazonas - Desemprego
0,483.900236,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,24.300584,1008.848463,...,21.357384,73.888459,1.602501,0.669899,3.692839,1.317344e+08,8.384593e+06,8.566149,1.216359e+08,8.630942
1,522.456171,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,21.294803,1009.316342,...,21.335315,71.475857,1.699240,0.673628,3.775586,1.336785e+08,8.472829e+06,8.602876,1.223016e+08,8.558110
2,569.481605,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,19.335488,1009.624655,...,22.328906,71.485298,1.805776,0.677007,3.824208,1.354495e+08,8.522905e+06,8.631972,1.228206e+08,8.526727
3,633.857084,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,18.298188,1008.796616,...,24.960940,75.718102,1.930499,0.680049,3.940207,1.370423e+08,8.592312e+06,8.653845,1.232167e+08,8.446276
4,731.905950,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,16.610190,1008.221260,...,25.178638,78.437621,1.812937,0.682761,4.044809,1.385256e+08,8.640798e+06,8.676548,1.235991e+08,8.434407
5,874.866956,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,19.314465,1008.202617,...,23.042487,77.493797,1.586819,0.685154,4.100371,1.396949e+08,8.691699e+06,8.687931,1.237178e+08,8.396886
6,1028.843987,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,20.202505,1007.958668,...,22.350784,78.022515,1.421020,0.687239,4.185427,1.406176e+08,8.722094e+06,8.695651,1.237698e+08,8.420659
7,1146.660535,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,20.155807,1008.053425,...,21.719583,78.179391,1.422041,0.689024,4.338533,1.439300e+08,8.745548e+06,8.730895,1.244348e+08,8.389077
8,1256.520719,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,20.048279,1008.445156,...,21.470218,77.933380,1.514544,0.690520,4.536829,1.469771e+08,8.709535e+06,8.759313,1.249553e+08,8.376520
9,1339.122148,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,20.344237,1008.651747,...,21.547865,78.542856,1.555217,0.691525,4.749958,1.498366e+08,8.659814e+06,8.775083,1.254646e+08,11.321769


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1733.557
1     1668.001
2     1934.967
3     2259.804
4     2678.337
5     2825.491
6     3349.109
7     3615.893
8     3744.322
9     3932.905
10    4055.140
11    3732.552
12    3424.968
13    3063.300
14    2990.152
15    3245.585
16    3803.900
17    3862.535
18    3948.984
19         NaN
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Bahia - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Amazonas - Desemprego
0,-1.777633,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,2.013063,0.014152,...,-0.574799,-1.175212,0.142976,-2.000203,-1.314053,-1.640868,-0.720159,-2.092923,-2.030446,-0.844534
1,-1.644610,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,0.563422,0.965443,...,-0.593143,-2.265111,0.811401,-1.607485,-1.204074,-1.419829,-0.214461,-1.618766,-1.566716,-0.879556
2,-1.482367,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.381525,1.592305,...,0.232709,-2.260846,1.547511,-1.251484,-1.139450,-1.218469,0.072528,-1.243121,-1.205123,-0.894648
3,-1.260263,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.881798,-0.091264,...,2.420396,-0.348666,2.409292,-0.931159,-0.985276,-1.037371,0.470310,-0.960725,-0.929213,-0.933333
4,-0.921982,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-1.695893,-1.261075,...,2.601342,0.879885,1.596989,-0.645467,-0.846249,-0.868728,0.748190,-0.667610,-0.662847,-0.939040
5,-0.428748,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.391663,-1.298980,...,0.825821,0.453510,0.034621,-0.393366,-0.772402,-0.735775,1.039914,-0.520646,-0.580119,-0.957083
6,0.102493,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,0.036624,-1.794978,...,0.250893,0.692359,-1.110974,-0.173813,-0.659353,-0.630868,1.214113,-0.420985,-0.543919,-0.945651
7,0.508975,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.014102,-1.602318,...,-0.273747,0.763229,-1.103919,0.014235,-0.455860,-0.254262,1.348531,0.034040,-0.080622,-0.960838
8,0.888007,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.037756,-0.805851,...,-0.481014,0.652093,-0.464761,0.171819,-0.192305,0.092186,1.142132,0.400926,0.281975,-0.966876
9,1.172993,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.104979,-0.385810,...,-0.416476,0.927425,-0.183732,0.277727,0.090965,0.417297,0.857173,0.604537,0.636757,0.449384


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1733.557
1     1668.001
2     1934.967
3     2259.804
4     2678.337
5     2825.491
6     3349.109
7     3615.893
8     3744.322
9     3932.905
10    4055.140
11    3732.552
12    3424.968
13    3063.300
14    2990.152
15    3245.585
16    3803.900
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Bahia - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Amazonas - Desemprego
17,0.693426,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.843886,0.014115,...,-0.383107,0.758267,-1.258928,0.942649,2.009168,0.780976,-1.283382,0.459812,0.390733,0.934453


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    3862.535
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [12]:
train_input.iloc[-(len(train_input)//5):]

,Bahia - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Amazonas - Desemprego
14,0.377747,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,0.318622,0.827708,...,-0.719961,-0.207541,-0.705978,1.337347,1.318946,1.196726,-1.494416,1.022881,0.982310,1.170343
15,0.450830,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.277647,0.596013,...,-0.515322,0.113478,-0.965994,1.256770,1.539168,1.155428,-1.575619,1.044361,0.865889,1.093983
16,0.590131,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,1.933417,0.349463,...,-0.425121,0.484960,-1.074434,1.168867,1.812779,1.021400,-1.526499,0.744762,0.634724,1.052576


In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [14]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1396093451, 464761003, 4281133321, 445785743, 884463000, 3577841447, 3770125923, 66033198, 2391578507, 4045497973, 655044174, 3014141092, 989687679, 2977232205, 2282240619, 1781030640, 2470308725, 3980601888, 3476242912, 3416639815, 1358491953, 3756721014, 1341471282, 695632994, 2920791152, 850414206, 452522967, 2248317024, 3123522444, 1525194064, 1820252812, 3733030165, 3229819203, 3263121416, 5374389, 3498163105, 3159250578, 1595976959, 2554157560, 2621605361, 1576518907, 2920760457, 2371605288, 907675196, 579698573, 4225586865, 24276671, 4017194042, 3376415496, 1855469308, 3661970654, 1922802669, 2951705563, 4101176123, 3573637700, 4090866472, 141919492, 3607659560, 3599758610, 2436122505, 1386819978, 3373752805, 1909545476, 737161807, 2733174796, 2794733186, 1622540832, 1137433441, 334981211, 1215071369, 2252912241, 421596847, 1330708841, 2891553628, 599724435, 518352107, 540600282, 35938746, 3641360991, 2318303688, 530050965, 433833935, 2460775259, 1198672865, 906370774, 11460996

val_loss: 109799.8359375


Step: 88 ___________________________________________
val_loss: 91090.6640625


Step: 89 ___________________________________________
val_loss: 115460.7734375


Step: 90 ___________________________________________
val_loss: 107964.3671875


Step: 91 ___________________________________________
val_loss: 115460.7890625


Step: 92 ___________________________________________
val_loss: 108636.7890625


Step: 93 ___________________________________________
val_loss: 69929.3125


Step: 94 ___________________________________________
val_loss: 115462.5234375


Step: 95 ___________________________________________
val_loss: 124683.375


Step: 96 ___________________________________________
val_loss: 97993.6796875


Step: 97 ___________________________________________
val_loss: 40709.51171875


Step: 98 ___________________________________________
val_loss: 113943.7109375


Step: 99 ___________________________________________
val_loss: 115460.7890625


final_seed: 24276671


In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 460ms/step - loss: 9942037.0000 - val_loss: 11210165.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 9783586.0000 - val_loss: 13366052.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 11142597.0000 - val_loss: 12128901.0000
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 9547089.0000 - val_loss: 4280049.5000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 6736456.5000 - val_loss: 4246127.5000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 5750522.5000 - val_loss: 2244001.2500
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 6162614.5000 - val_loss: 3398962.7500
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 5928919.0000 - val_loss: 1799560.3750
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 5201456.0000 - val_loss: 

1/1 [==============================] - 0s 14ms/step - loss: 2178769.2500 - val_loss: 837826.7500
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 1622998.5000 - val_loss: 560844.0000
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 1732866.5000 - val_loss: 727773.5000
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 1953882.0000 - val_loss: 848081.8125
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 2152087.2500 - val_loss: 641647.6875
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 2189429.5000 - val_loss: 1157792.6250
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 2745021.2500 - val_loss: 1440206.1250
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 2016634.3750 - val_loss: 947591.8125
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 2273866.7500 - val_loss: 1077265.7500
Epoc

1/1 [==============================] - 0s 14ms/step - loss: 791780.8125 - val_loss: 307594.5000
Epoch 147/10000
1/1 [==============================] - 0s 13ms/step - loss: 598674.6875 - val_loss: 185372.0781
Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 606067.6250 - val_loss: 115122.1016
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 766880.8750 - val_loss: 302643.5312
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 956017.0625 - val_loss: 361577.0312
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 554988.3125 - val_loss: 225383.6719
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 572908.3750 - val_loss: 159840.4219
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 520319.5312 - val_loss: 217634.6875
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 553858.3125 - val_loss: 194805.2031
Epoch 15

1/1 [==============================] - 0s 14ms/step - loss: 272772.3125 - val_loss: 106403.6562
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 366660.1875 - val_loss: 114723.5000
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 266979.6875 - val_loss: 99371.9141
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 204145.3125 - val_loss: 79770.1719
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 393143.7812 - val_loss: 121902.9609
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 208933.3906 - val_loss: 80965.6719
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 243375.5156 - val_loss: 105147.6562
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 333365.1250 - val_loss: 109812.3516
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 337132.4688 - val_loss: 125059.6562
Epoch 228/1

Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 113784.0391 - val_loss: 34621.3242
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 131627.6719 - val_loss: 42683.4883
Epoch 295/10000
1/1 [==============================] - 0s 13ms/step - loss: 148907.8281 - val_loss: 38279.1797
Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 141764.8750 - val_loss: 37112.0156
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 138718.0469 - val_loss: 47414.0977
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 128803.2500 - val_loss: 35889.8555
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 113445.8906 - val_loss: 39858.7773
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 93146.1016 - val_loss: 39719.6094
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 144431.7188 - val_loss: 41822.6445
Ep

...

1/1 [==============================] - 0s 14ms/step - loss: 87905.0781 - val_loss: 62684.2500
Epoch 886/10000
1/1 [==============================] - 0s 15ms/step - loss: 86959.0156 - val_loss: 60712.2773
Epoch 887/10000
1/1 [==============================] - 0s 14ms/step - loss: 108131.3984 - val_loss: 52203.8867
Epoch 888/10000
1/1 [==============================] - 0s 14ms/step - loss: 49415.7852 - val_loss: 51032.4219
Epoch 889/10000
1/1 [==============================] - 0s 14ms/step - loss: 98586.1250 - val_loss: 60367.0508
Epoch 890/10000
1/1 [==============================] - 0s 14ms/step - loss: 95959.8047 - val_loss: 52564.8633
Epoch 891/10000
1/1 [==============================] - 0s 13ms/step - loss: 87905.4766 - val_loss: 55320.9883
Epoch 892/10000
1/1 [==============================] - 0s 13ms/step - loss: 98826.4688 - val_loss: 52895.1875
Epoch 893/10000
1/1 [==============================] - 0s 13ms/step - loss: 85831.6875 - val_loss: 56500.4883
Epoch 894/10000
1/1 [====

1/1 [==============================] - 0s 14ms/step - loss: 49270.1953 - val_loss: 84504.8672
Epoch 960/10000
1/1 [==============================] - 0s 15ms/step - loss: 74593.5234 - val_loss: 90550.1641
Epoch 961/10000
1/1 [==============================] - 0s 15ms/step - loss: 89119.3516 - val_loss: 110404.3359
Epoch 962/10000
1/1 [==============================] - 0s 14ms/step - loss: 98503.9766 - val_loss: 86541.7422
Epoch 963/10000
1/1 [==============================] - 0s 14ms/step - loss: 51777.7617 - val_loss: 88971.8203
Epoch 964/10000
1/1 [==============================] - 0s 33ms/step - loss: 58554.2734 - val_loss: 80895.5859
Epoch 965/10000
1/1 [==============================] - 0s 14ms/step - loss: 60071.6719 - val_loss: 82187.9609
Epoch 966/10000
1/1 [==============================] - 0s 14ms/step - loss: 73414.5000 - val_loss: 90791.0547
Epoch 967/10000
1/1 [==============================] - 0s 14ms/step - loss: 65695.0703 - val_loss: 85439.5859
Epoch 968/10000
1/1 [====

1/1 [==============================] - 0s 14ms/step - loss: 77718.2656 - val_loss: 93093.2500
Epoch 1034/10000
1/1 [==============================] - 0s 14ms/step - loss: 56683.5781 - val_loss: 89753.0625
Epoch 1035/10000
1/1 [==============================] - 0s 14ms/step - loss: 54854.1914 - val_loss: 88135.3125
Epoch 1036/10000
1/1 [==============================] - 0s 14ms/step - loss: 71496.2266 - val_loss: 94278.3125
Epoch 1037/10000
1/1 [==============================] - 0s 14ms/step - loss: 39843.5938 - val_loss: 93427.3672
Epoch 1038/10000
1/1 [==============================] - 0s 14ms/step - loss: 89454.6875 - val_loss: 87550.6875
Epoch 1039/10000
1/1 [==============================] - 0s 13ms/step - loss: 87847.6328 - val_loss: 96333.9141
Epoch 1040/10000
1/1 [==============================] - 0s 14ms/step - loss: 59882.0352 - val_loss: 101479.6641
Epoch 1041/10000
1/1 [==============================] - 0s 13ms/step - loss: 54202.3633 - val_loss: 98634.6875
Epoch 1042/10000


Epoch 1107/10000
1/1 [==============================] - 0s 14ms/step - loss: 60662.9414 - val_loss: 72337.5859
Epoch 1108/10000
1/1 [==============================] - 0s 14ms/step - loss: 54589.2930 - val_loss: 68329.2109
Epoch 1109/10000
1/1 [==============================] - 0s 13ms/step - loss: 58256.3320 - val_loss: 66509.4922
Epoch 1110/10000
1/1 [==============================] - 0s 13ms/step - loss: 86127.3047 - val_loss: 69931.1328
Epoch 1111/10000
1/1 [==============================] - 0s 13ms/step - loss: 101298.8359 - val_loss: 72933.6953
Epoch 1112/10000
1/1 [==============================] - 0s 13ms/step - loss: 89546.1328 - val_loss: 76820.9609
Epoch 1113/10000
1/1 [==============================] - 0s 13ms/step - loss: 68216.1172 - val_loss: 71054.7344
Epoch 1114/10000
1/1 [==============================] - 0s 14ms/step - loss: 72431.9375 - val_loss: 67924.5938
Epoch 1115/10000
1/1 [==============================] - 0s 14ms/step - loss: 81263.6953 - val_loss: 72747.1797


1/1 [==============================] - 0s 13ms/step - loss: 48490.4883 - val_loss: 49756.3320
Epoch 1181/10000
1/1 [==============================] - 0s 13ms/step - loss: 62064.6484 - val_loss: 32557.8730
Epoch 1182/10000
1/1 [==============================] - 0s 13ms/step - loss: 92717.0781 - val_loss: 31709.9395
Epoch 1183/10000
1/1 [==============================] - 0s 14ms/step - loss: 103884.8906 - val_loss: 43415.1758
Epoch 1184/10000
1/1 [==============================] - 0s 14ms/step - loss: 45787.0898 - val_loss: 41017.0664
Epoch 1185/10000
1/1 [==============================] - 0s 13ms/step - loss: 36002.5352 - val_loss: 38101.8008
Epoch 1186/10000
1/1 [==============================] - 0s 14ms/step - loss: 131694.7031 - val_loss: 26917.9531
Epoch 1187/10000
1/1 [==============================] - 0s 13ms/step - loss: 105265.2109 - val_loss: 37902.6719
Epoch 1188/10000
1/1 [==============================] - 0s 14ms/step - loss: 94998.4688 - val_loss: 27977.7871
Epoch 1189/1000

In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[3862.535](test_target) - [[3687.1091]](prediction) = 175.42586914062485


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Bahia - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Amazonas - Desemprego
0,-1.157436,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,2.070335,0.382130,...,-1.002645,-0.615132,-0.331368,-1.699744,-1.432505,-1.583772,-1.770958,-1.787213,-1.841951,1.914585
1,-0.989097,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,0.643565,1.192212,...,-1.018153,-1.488540,0.225135,-1.107241,-1.030565,-1.057420,-1.033882,-1.054519,-1.051473,1.018176
2,-0.783779,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.286474,1.726022,...,-0.319968,-1.485122,0.837991,-0.570134,-0.794386,-0.577928,-0.615583,-0.474052,-0.435098,0.631905
3,-0.502710,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.778855,0.292363,...,1.529530,0.047233,1.555475,-0.086852,-0.230927,-0.146685,-0.035798,-0.037678,0.035222,-0.358282
4,-0.074619,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-1.580106,-0.703802,...,1.682504,1.031751,0.879184,0.344178,0.277174,0.254898,0.369224,0.415259,0.489272,-0.504360
5,0.549562,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.296453,-0.736081,...,0.181457,0.690069,-0.421580,0.724530,0.547062,0.571495,0.794424,0.642357,0.630291,-0.966173
6,1.221840,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.125078,-1.158453,...,-0.304594,0.881474,-1.375356,1.055776,0.960220,0.821306,1.048327,0.796358,0.691998,-0.673571
7,1.736238,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,0.102911,-0.994391,...,-0.748132,0.938267,-1.369482,1.339487,1.703926,1.718106,1.244246,1.499489,1.481740,-1.062282


train_target:


,Bahia - Consumo de Cimento (t)
0,1733.557
1,1668.001
2,1934.967
3,2259.804
4,2678.337
5,2825.491
6,3349.109
7,3615.893


test_input:


...

,Bahia - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Bahia - IDH,Bahia - Valor Cimento Portland,Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Amazonas - Desemprego
17,0.693426,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.843886,0.014115,...,-0.383107,0.758267,-1.258928,0.942649,2.009168,0.780976,-1.283382,0.459812,0.390733,0.934453


test_target:


,Bahia - Consumo de Cimento (t)
17,3862.535


1/1 [==============================] - 0s 35ms/step
Error: 175.42586914062485




[3423.62158203125,
 3466.4716796875,
 3671.89306640625,
 3920.3173828125,
 3763.669189453125,
 3687.80322265625,
 3296.35595703125,
 3746.412841796875,
 3283.066650390625,
 3687.109130859375]

In [21]:
display(targets)
display(predictions)

[3744.322,
 3932.905,
 4055.14,
 3732.551999999999,
 3424.967999999999,
 3063.3,
 2990.152,
 3245.585,
 3803.9,
 3862.535]

[3423.62158203125,
 3466.4716796875,
 3671.89306640625,
 3920.3173828125,
 3763.669189453125,
 3687.80322265625,
 3296.35595703125,
 3746.412841796875,
 3283.066650390625,
 3687.109130859375]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

382.46414843750017

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.10666861498653527